<a href="https://colab.research.google.com/github/Poojads-udupi/AI--Residency-Gradio/blob/main/story_telling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Step 1: Install required packages
!pip install --upgrade openai gradio fpdf gTTS

# 📌 Step 2: Import Libraries
from google.colab import userdata
import os
import gradio as gr
from openai import OpenAI
from fpdf import FPDF
from datetime import datetime
from gtts import gTTS

# 📌 Step 3: Load API Key securely
api_key = userdata.get("openai")
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

# 📌 Step 4: Directory to save stories
story_dir = "/content/storybot_stories"
os.makedirs(story_dir, exist_ok=True)

# 📌 Step 5: Save story to PDF
def save_story_as_pdf(story_text, character_name=""):
    date_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{character_name or 'story'}_{date_str}.pdf"
    file_path = os.path.join(story_dir, file_name)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt=f"{character_name}'s Adventure", ln=True, align='C')
    pdf.set_font("Arial", size=12)
    pdf.ln(10)

    for line in story_text.split('\n'):
        pdf.multi_cell(0, 10, line)

    pdf.add_page()
    pdf.set_font("Arial", 'I', 14)
    pdf.cell(200, 10, txt="The End", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(200, 10, txt="Created with Magic StoryBot", ln=True, align='C')

    pdf.output(file_path)
    return file_path

# 📌 Step 6: Convert story to speech
def text_to_speech(text):
    tts = gTTS(text)
    audio_path = "/content/story_audio.mp3"
    tts.save(audio_path)
    return audio_path

# 📌 Step 7: Generate story from LLM
def generate_story(age_group, theme, character_name, tone, length):
    length_instruction = {
        "Short": "about 100 words",
        "Medium": "about 200 words",
        "Long": "about 350 words"
    }.get(length, "about 150 words")

    system_prompt = f"""
You are a children's storyteller. Create a {length_instruction}, age-appropriate story for a child aged {age_group}.
Use the theme "{theme}" and the main character's name is "{character_name}".
Make the tone of the story {tone.lower()}. End it with the phrase "The End".
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Please generate the story now."}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            stop=None
        )
        print("LLM Response:", response)  # Debug print
        story = response.choices[0].message.content
    except Exception as e:
        print("LLM ERROR:", e)
        story = f"An error occurred: {str(e)}"

    audio_path = text_to_speech(story)
    return story, story, audio_path

# 📌 Step 8: Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 📖 Magic StoryBot for Kids")

    age_group = gr.Dropdown(["1–3", "3–5", "6–8", "9–12"], label="Select Age Group")
    theme = gr.Dropdown(["Animals", "Adventure", "Magic", "Mystery", "Bedtime"], label="Choose a Story Theme")
    character_name = gr.Textbox(label="Main Character's Name", placeholder="e.g., Luna, Max")
    tone = gr.Dropdown(["Happy", "Funny", "Gentle", "Spooky", "Exciting", "Calm"], label="Story Tone")
    length = gr.Radio(["Short", "Medium", "Long"], label="Story Length")

    story_output = gr.Textbox(label="Generated Story", lines=10)
    story_audio = gr.Audio(label="Listen", autoplay=True)
    pdf_output = gr.File(label="Download Your Story", file_types=[".pdf"], visible=True)

    def start_story(age, theme, name, tone, length):
        story, _, audio = generate_story(age, theme, name, tone, length)
        return story, audio

    def export_pdf(story, name):
        pdf_path = save_story_as_pdf(story, name)
        return pdf_path

    start_btn = gr.Button("🌟 Generate Story")
    save_btn = gr.Button("📥 Save Story as PDF")

    start_btn.click(start_story,
                    inputs=[age_group, theme, character_name, tone, length],
                    outputs=[story_output, story_audio])

    save_btn.click(export_pdf, inputs=[story_output, character_name], outputs=pdf_output)

app.launch()




It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e581b405a25aa16627.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 📌 Step 1: Install required packages
!pip install --upgrade openai gradio fpdf gTTS diffusers transformers accelerate safetensors

# 📌 Step 2: Import Libraries
from google.colab import userdata
import os
import gradio as gr
import openai
import requests
from fpdf import FPDF
from datetime import datetime
from gtts import gTTS
from diffusers import StableDiffusionPipeline
import torch

# 📌 Step 3: Load API Key securely
api_key = userdata.get("openai")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

# 📌 Step 4: Load Hugging Face image model (free-tier)
hf_model = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16,
    revision="fp16"
).to("cuda" if torch.cuda.is_available() else "cpu")

# 📌 Step 5: Create story directory
story_dir = "/content/storybot_stories"
os.makedirs(story_dir, exist_ok=True)

# 📌 Step 6: Save story to PDF
def save_story_as_pdf(story_text, character_name=""):
    date_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{character_name or 'story'}_{date_str}.pdf"
    file_path = os.path.join(story_dir, file_name)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt=f"{character_name}'s Adventure", ln=True, align='C')
    pdf.set_font("Arial", size=12)
    pdf.ln(10)

    for line in story_text.split('\n'):
        pdf.multi_cell(0, 10, line)

    pdf.add_page()
    pdf.set_font("Arial", 'I', 14)
    pdf.cell(200, 10, txt="The End", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(200, 10, txt="Created with Magic StoryBot", ln=True, align='C')

    pdf.output(file_path)
    return file_path

# 📌 Step 7: Convert story to speech
def text_to_speech(text):
    tts = gTTS(text)
    audio_path = "/content/story_audio.mp3"
    tts.save(audio_path)
    return audio_path

# 📌 Step 8: Generate story and image
def generate_story(age_group, theme, character_name, tone, length):
    length_instruction = {
        "Short": "about 100 words",
        "Medium": "about 200 words",
        "Long": "about 350 words"
    }.get(length, "about 150 words")

    system_prompt = f"""
You are a children's storyteller. Create a {length_instruction}, age-appropriate story for a child aged {age_group}.
Use the theme "{theme}" and the main character's name is "{character_name}".
Make the tone of the story {tone.lower()}. End it with the phrase "The End".
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Please generate the story now."}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=1024
        )
        story = response.choices[0].message.content
        print("LLM Response:", story)
    except Exception as e:
        print("LLM ERROR:", e)
        story = f"An error occurred: {str(e)}"

    # 🎨 Generate image using Hugging Face model
    image_prompt = f"A {tone.lower()} illustration for a children's story about {character_name} and the theme of {theme}."
    try:
        image = hf_model(image_prompt).images[0]
        image_path = "/content/story_image.png"
        image.save(image_path)
    except Exception as e:
        print("Image generation error:", e)
        image_path = None

    audio_path = text_to_speech(story)
    return story, audio_path, image_path

# 📌 Step 9: Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 📖 Magic StoryBot for Kids")

    age_group = gr.Dropdown(["1–3", "3–5", "6–8", "9–12"], label="Select Age Group")
    theme = gr.Dropdown(["Animals", "Adventure", "Magic", "Mystery", "Bedtime"], label="Choose a Story Theme")
    character_name = gr.Textbox(label="Main Character's Name", placeholder="e.g., Luna, Max")
    tone = gr.Dropdown(["Happy", "Funny", "Gentle", "Spooky", "Exciting", "Calm"], label="Story Tone")
    length = gr.Radio(["Short", "Medium", "Long"], label="Story Length")

    story_output = gr.Textbox(label="Generated Story", lines=10)
    story_audio = gr.Audio(label="Listen", autoplay=True)
    story_image = gr.Image(label="Story Illustration", type="filepath")
    pdf_output = gr.File(label="Download Your Story", file_types=[".pdf"], visible=True)

    def start_story(age, theme, name, tone, length):
        story, audio, image_path = generate_story(age, theme, name, tone, length)
        return story, audio, image_path

    def export_pdf(story, name):
        pdf_path = save_story_as_pdf(story, name)
        return pdf_path

    start_btn = gr.Button("🌟 Generate Story")
    save_btn = gr.Button("📥 Save Story as PDF")

    start_btn.click(start_story,
                    inputs=[age_group, theme, character_name, tone, length],
                    outputs=[story_output, story_audio, story_image])

    save_btn.click(export_pdf, inputs=[story_output, character_name], outputs=pdf_output)

app.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 784.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:285: FutureWarning: You are loading the variant fp16 from stabilityai/stable-diffusion-2-1 via `revision='fp16'`. This behavior is deprecated and will be removed in diffusers v1. One should use `variant='fp16'` instead. However, it appears that stabilityai/stable-diffusion-2-1 currently does not have the required variant filenames in the 'main' branch. 
 The Diffusers team and community would be very grateful if you could open an issue: https://github.com/huggingface/diffusers/issues/new with the title 'stabilityai/stable-diffusion-2-1 is missing fp16 files' so that the correct variant file can be added.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/f7f33030acc57428be85fbec092c37a78231d75a/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is no

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83fe93ec5ddd6e74dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 📌 Step 1: Install required packages
!pip install --upgrade openai gradio fpdf gTTS

# 📌 Step 2: Import Libraries
from google.colab import userdata
import os
import gradio as gr
import openai
import requests
from fpdf import FPDF
from datetime import datetime
from gtts import gTTS

# 📌 Step 3: Load API Key securely
api_key = userdata.get("openai")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

# 📌 Step 4: Directory to save stories
story_dir = "/content/storybot_stories"
os.makedirs(story_dir, exist_ok=True)

# 📌 Step 5: Save story to PDF
def save_story_as_pdf(story_text, character_name=""):
    date_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{character_name or 'story'}_{date_str}.pdf"
    file_path = os.path.join(story_dir, file_name)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt=f"{character_name}'s Adventure", ln=True, align='C')
    pdf.set_font("Arial", size=12)
    pdf.ln(10)

    for line in story_text.split('\n'):
        pdf.multi_cell(0, 10, line)

    pdf.add_page()
    pdf.set_font("Arial", 'I', 14)
    pdf.cell(200, 10, txt="The End", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(200, 10, txt="Created with Magic StoryBot", ln=True, align='C')

    pdf.output(file_path)
    return file_path

# 📌 Step 6: Convert story to speech
def text_to_speech(text):
    tts = gTTS(text)
    audio_path = "/content/story_audio.mp3"
    tts.save(audio_path)
    return audio_path

# 📌 Step 7: Generate story + image
def generate_story(age_group, theme, character_name, tone, length):
    length_instruction = {
        "Short": "about 100 words",
        "Medium": "about 200 words",
        "Long": "about 350 words"
    }.get(length, "about 150 words")

    system_prompt = f"""
You are a children's storyteller. Create a {length_instruction}, age-appropriate story for a child aged {age_group}.
Use the theme "{theme}" and the main character's name is "{character_name}".
Make the tone of the story {tone.lower()}. End it with the phrase "The End".
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Please generate the story now."}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=1024
        )
        story = response.choices[0].message.content
        print("LLM Response:", story)
    except Exception as e:
        print("LLM ERROR:", e)
        story = f"An error occurred: {str(e)}"

    # 📸 Generate Image using DALL·E 2
    image_prompt = f"A {tone.lower()} illustration for a children's story about {character_name} and the theme of {theme}."
    try:
        dalle_response = openai.Image.create(
            prompt=image_prompt,
            n=1,
            size="512x512"
        )
        image_url = dalle_response['data'][0]['url']
        image_path = "/content/story_image.png"
        with open(image_path, "wb") as f:
            f.write(requests.get(image_url).content)
    except Exception as e:
        print("Image generation error:", e)
        image_path = None

    audio_path = text_to_speech(story)
    return story, audio_path, image_path

# 📌 Step 8: Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 📖 Magic StoryBot for Kids")

    age_group = gr.Dropdown(["1–3", "3–5", "6–8", "9–12"], label="Select Age Group")
    theme = gr.Dropdown(["Animals", "Adventure", "Magic", "Mystery", "Bedtime"], label="Choose a Story Theme")
    character_name = gr.Textbox(label="Main Character's Name", placeholder="e.g., Luna, Max")
    tone = gr.Dropdown(["Happy", "Funny", "Gentle", "Spooky", "Exciting", "Calm"], label="Story Tone")
    length = gr.Radio(["Short", "Medium", "Long"], label="Story Length")

    story_output = gr.Textbox(label="Generated Story", lines=10)
    story_audio = gr.Audio(label="Listen", autoplay=True)
    story_image = gr.Image(label="Story Illustration", type="filepath")
    pdf_output = gr.File(label="Download Your Story", file_types=[".pdf"], visible=True)

    def start_story(age, theme, name, tone, length):
        story, audio, image_path = generate_story(age, theme, name, tone, length)
        return story, audio, image_path

    def export_pdf(story, name):
        pdf_path = save_story_as_pdf(story, name)
        return pdf_path

    start_btn = gr.Button("🌟 Generate Story")
    save_btn = gr.Button("📥 Save Story as PDF")

    start_btn.click(start_story,
                    inputs=[age_group, theme, character_name, tone, length],
                    outputs=[story_output, story_audio, story_image])

    save_btn.click(export_pdf, inputs=[story_output, character_name], outputs=pdf_output)

app.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e0b6f57c66ac5df915.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip show openai

Name: openai
Version: 1.79.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
